In [ ]:
import pandas as pd
import json
import random



In [ ]:
!pip install requests



In [ ]:
import requests

In [ ]:
!pip install --upgrade google-generativeai


In [ ]:
df = pd.read_csv("yelp.csv.zip")
df.head()


,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [ ]:
df.columns


Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny'],
      dtype='object')

In [ ]:
df_clean = df[['text', 'stars']].copy()
df_clean.head()


,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [ ]:
df_clean.rename(
    columns={
        'text': 'review_text',
        'stars': 'actual_stars'
    },
    inplace=True
)

df_clean.head()


,review_text,actual_stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [ ]:
df_sample = df_clean.sample(n=200, random_state=42)
df_sample.reset_index(drop=True, inplace=True)

df_sample.head()


,review_text,actual_stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5


In [ ]:
df_sample['actual_stars'].value_counts().sort_index()


,count
actual_stars,
1,18
2,17
3,33
4,79
5,53


In [ ]:
PROMPT_V1 = """
Classify the following Yelp review into a star rating from 1 to 5.

Return ONLY valid JSON in this format:
{{
  "predicted_stars": number,
  "explanation": "short explanation"
}}

Review:
{review}
"""


In [ ]:
def build_prompt(prompt_template, review_text):
    return prompt_template.format(review=review_text)


In [ ]:
sample_review = df_sample.loc[0, 'review_text']

prompt = build_prompt(PROMPT_V1, sample_review)
print(prompt)



Classify the following Yelp review into a star rating from 1 to 5.

Return ONLY valid JSON in this format:
{
  "predicted_stars": number,
  "explanation": "short explanation"
}

Review:
We got here around midnight last Friday... the place was dead. However, they were still serving food and we enjoyed some well made pub grub. Service was friendly, quality cocktails were served, and the atmosphere is derived from an old Uno's, which certainly works for a sports bar. It being located in a somewhat commercial area, I can see why it's empty so late on a Friday. From what my friends tell me - this is a great spot for happy hour, and it stays relatively busy thru 10pm.

*UPDATE - Great patio for day-drinking on the weekends!



In [ ]:
df_sample.columns


Index(['review_text', 'actual_stars'], dtype='object')

In [ ]:
OPENROUTER_API_KEY = "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"


In [ ]:
response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "mistralai/mistral-7b-instruct",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
)

print(response.json())


{'id': 'gen-1767723485-Ilfb8PSMtbaLWbY5g1C6', 'provider': 'DeepInfra', 'model': 'mistralai/mistral-7b-instruct', 'object': 'chat.completion', 'created': 1767723485, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': ' <s> {"predicted_stars": 4, "explanation": "Positive review with compliments on food, service, and atmosphere, despite noting it was empty at midnight. The update adds a positive note about the patio."}', 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 185, 'completion_tokens': 50, 'total_tokens': 235, 'cost': 7.88e-06, 'is_byok': False, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0, 'video_tokens': 0}, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 5.18e-06, 'upstream_inference_completions_cost': 2.7e-06}, 'completion_tokens_details': {'reasoning_tokens': 0, 'image_tokens': 0}}}


In [ ]:
response_json = response.json()
model_output = response_json["choices"][0]["message"]["content"]

print(model_output)


 <s> {"predicted_stars": 4, "explanation": "Positive review with compliments on food, service, and atmosphere, despite noting it was empty at midnight. The update adds a positive note about the patio."}


In [ ]:

def parse_model_output(text):
    try:
        parsed = json.loads(text)
        return parsed, True
    except Exception as e:
        return None, False


In [ ]:
parsed_output, is_valid = parse_model_output(model_output)

print("JSON valid:", is_valid)
print("Parsed output:", parsed_output)



JSON valid: False
Parsed output: None


In [ ]:
predicted_stars_v1 = []
json_valid_v1 = []
raw_outputs_v1 = []


In [ ]:
for idx, row in df_sample.iterrows():
    review_text = row["review_text"]

    prompt = build_prompt(PROMPT_V1, review_text)

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": "mistralai/mistral-7b-instruct",
                "messages": [
                    {"role": "user", "content": prompt}
                ]
            }
        )

        response_json = response.json()
        model_text = response_json["choices"][0]["message"]["content"]

        parsed, is_valid = parse_model_output(model_text)

        if is_valid and "predicted_stars" in parsed:
            predicted_stars_v1.append(parsed["predicted_stars"])
        else:
            predicted_stars_v1.append(None)

        json_valid_v1.append(is_valid)
        raw_outputs_v1.append(model_text)

    except Exception as e:
        predicted_stars_v1.append(None)
        json_valid_v1.append(False)
        raw_outputs_v1.append(None)


In [ ]:
df_sample["predicted_stars_v1"] = predicted_stars_v1
df_sample["json_valid_v1"] = json_valid_v1
df_sample["raw_output_v1"] = raw_outputs_v1

df_sample.head()


,review_text,actual_stars,predicted_stars_v1,json_valid_v1,raw_output_v1
0,We got here around midnight last Friday... the...,4,4.0,True,"\n\n{\n ""predicted_stars"": 4,\n ""explanatio..."
1,Brought a friend from Louisiana here. She say...,5,4.5,True,"{\n ""predicted_stars"": 4.5,\n ""explanation""..."
2,"Every friday, my dad and I eat here. We order ...",3,NaN,False,
3,"My husband and I were really, really disappoin...",1,NaN,False,"<s> {""predicted_stars"": 1, ""explanation"": ""Th..."
4,Love this place! Was in phoenix 3 weeks for w...,5,NaN,False,"<s> {\n ""predicted_stars"": 5,\n ""explanatio..."


In [ ]:
json_valid_rate_v1 = df_sample["json_valid_v1"].mean()
json_valid_rate_v1


np.float64(0.18)

In [ ]:
valid_rows = df_sample[df_sample["predicted_stars_v1"].notna()]

accuracy_v1 = (
    valid_rows["predicted_stars_v1"].round().astype(int)
    == valid_rows["actual_stars"]
).mean()

accuracy_v1


np.float64(0.5833333333333334)

In [ ]:
PROMPT_V2 = """
You are a Yelp review rating classifier.

Rules:
- Return ONLY valid JSON
- Do NOT add any text outside JSON
- predicted_stars MUST be an integer from 1 to 5

JSON format:
{{
  "predicted_stars": 1,
  "explanation": "short reason"
}}

Review:
{review}
"""


In [ ]:
predicted_stars_v2 = []
json_valid_v2 = []
raw_outputs_v2 = []


In [ ]:
for idx, row in df_sample.iterrows():
    prompt = build_prompt(PROMPT_V2, row["review_text"])

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": "mistralai/mistral-7b-instruct",
                "messages": [
                    {"role": "user", "content": prompt}
                ]
            }
        )

        response_json = response.json()
        model_text = response_json["choices"][0]["message"]["content"]
        parsed, is_valid = parse_model_output(model_text)

        if is_valid and "predicted_stars" in parsed:
            predicted_stars_v2.append(parsed["predicted_stars"])
        else:
            predicted_stars_v2.append(None)

        json_valid_v2.append(is_valid)
        raw_outputs_v2.append(model_text)

    except:
        predicted_stars_v2.append(None)
        json_valid_v2.append(False)
        raw_outputs_v2.append(None)


In [ ]:
df_sample["predicted_stars_v2"] = predicted_stars_v2
df_sample["json_valid_v2"] = json_valid_v2
df_sample["raw_output_v2"] = raw_outputs_v2


In [ ]:
json_valid_rate_v2 = df_sample["json_valid_v2"].mean()

valid_rows_v2 = df_sample[df_sample["predicted_stars_v2"].notna()]
accuracy_v2 = (
    valid_rows_v2["predicted_stars_v2"].astype(int)
    == valid_rows_v2["actual_stars"]
).mean()

json_valid_rate_v2, accuracy_v2


(np.float64(0.295), np.float64(0.6440677966101694))

In [ ]:
PROMPT_V3 = """
You are a strict Yelp review rating classifier.

Star rules:
1 = very bad, angry, complaints, dissatisfaction
2 = mostly negative, several issues
3 = mixed or neutral experience
4 = mostly positive, minor issues
5 = excellent, very satisfied, strong praise

Rules:
- Output ONLY valid JSON
- No extra text
- predicted_stars MUST be an integer (1–5)

JSON format:
{{
  "predicted_stars": 1,
  "explanation": "short reason"
}}

Review:
{review}
"""


In [ ]:
predicted_stars_v3 = []
json_valid_v3 = []
raw_outputs_v3 = []


In [ ]:
for idx, row in df_sample.iterrows():
    prompt = build_prompt(PROMPT_V3, row["review_text"])

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": "mistralai/mistral-7b-instruct",
                "messages": [
                    {"role": "user", "content": prompt}
                ]
            }
        )

        response_json = response.json()
        model_text = response_json["choices"][0]["message"]["content"]
        parsed, is_valid = parse_model_output(model_text)

        if is_valid and "predicted_stars" in parsed:
            predicted_stars_v3.append(parsed["predicted_stars"])
        else:
            predicted_stars_v3.append(None)

        json_valid_v3.append(is_valid)
        raw_outputs_v3.append(model_text)

    except:
        predicted_stars_v3.append(None)
        json_valid_v3.append(False)
        raw_outputs_v3.append(None)


In [ ]:
df_sample["predicted_stars_v3"] = predicted_stars_v3
df_sample["json_valid_v3"] = json_valid_v3
df_sample["raw_output_v3"] = raw_outputs_v3


In [ ]:
json_valid_rate_v3 = df_sample["json_valid_v3"].mean()

valid_rows_v3 = df_sample[df_sample["predicted_stars_v3"].notna()]
accuracy_v3 = (
    valid_rows_v3["predicted_stars_v3"].astype(int)
    == valid_rows_v3["actual_stars"]
).mean()

json_valid_rate_v3, accuracy_v3


(np.float64(0.165), np.float64(0.5757575757575758))

In [ ]:
comparison_df = pd.DataFrame({
    "Prompt": ["V1 (Naive)", "V2 (Strict JSON)", "V3 (Anchored)"],
    "JSON Validity Rate": [
        json_valid_rate_v1,
        json_valid_rate_v2,
        json_valid_rate_v3
    ],
    "Accuracy": [
        accuracy_v1,
        accuracy_v2,
        accuracy_v3
    ]
})

comparison_df


,Prompt,JSON Validity Rate,Accuracy
0,V1 (Naive),0.180,0.583333
1,V2 (Strict JSON),0.295,0.644068
2,V3 (Anchored),0.165,0.575758


In [ ]:
comparison_df.to_csv("prompt_comparison_results.csv", index=False)
